In [17]:
import folium
import pickle
def load_path_graph(path):
    with open('graph.pkl', 'rb') as f:
        graph = pickle.load(f)


    node_pairs = list(zip(path[:-1], path[1:]))
    lines = []
    for edge in graph:
        if (edge['u'], edge['v']) in node_pairs or  (edge['v'], edge['u']) in node_pairs:
            lines.append(edge['geometry'])
    return lines

#start = 2270143902; end = 1079387396
#start = 426882161; end = 1737223506
start = 1718165260; end = 8513026827

In [18]:
import numpy as np
import pandas as pd
import threading
from multiprocessing import Process,Lock,Manager

global dist,accu_time,for_loop_time,sort_time,mid_time,nodes_query_time,edges_query_time,dist,done,sort_time
global edges, nodes, heuristics, visited, previous,distance,lock1,lock2,incumbent_cost


def job():
    
    global dist,accu_time,for_loop_time,sort_time,mid_time,nodes_query_time,edges_query_time,dist,done,sort_time
    global edges, nodes, heuristics, visited, previous,distance,lock1,lock2,incumbent_cost

    
    distance.sort(key=lambda x: x[1]+x[2])
    if(distance == []):
        return
    s = distance[0][0]
    d = distance[0][1]
    if(s == end):
        done = 1
        if(incumbent_cost > d):
            d = incumbent_cost
        #print("done = ",done)
        return
    best_dist = distance[0]
    distance.pop(0)
    if(best_dist[1] + best_dist[2] > incumbent_cost):
        return
    #print(best_dist)
    #if nodes.at[s,'visited'] == 0:
    #    nodes.at[s,'visited'] = 1
    #    nodes.at[s,'previous'] = best_dist[3]

    try:
        edge = edges.loc[int(s),:]
    except:
        return
    if not isinstance(edge, pd.DataFrame): 
        edge = edge.to_frame().transpose() # 轉成 DataFrame

    for index,edg in edge.iterrows():
        e = int(edg['end'])
        if(nodes.at[e,'visited'] == 0):
            distance.append([e, d + edg['distance'], heuristics.at[e,str(end)] , s])
            nodes.at[e,'visited'] = 1
            nodes.at[e,'g'] = d + edg['distance']
            nodes.at[e,'previous'] = s
            dist = d + edg['distance']
            #if e == end:
            #    nodes.at[end,'previous'] = s
            #    break
        '''else:
            if(nodes.at[e,'g'] > d + edg['distance']):
                distance.append([e, d + edg['distance'], heuristics.at[e,str(end)] , s])
                nodes.at[e,'g'] = d + edg['distance']
                dist = d + edg['distance']
                nodes.at[e,'previous'] = s
            else:
                pass '''
        #lock2.release()




def astar(start = 2270143902, end = 1079387396):
    
    global dist,accu_time,for_loop_time,sort_time,mid_time,nodes_query_time,edges_query_time,dist,done,sort_time
    global edges, nodes, heuristics, visited, previous,distance,lock1,lock2,incumbent_cost
    
    edges = pd.read_csv('./edges.csv')
    edges.sort_values(by=['start'])
    edges_start_list = edges['start'].tolist()
    nodes = pd.DataFrame(edges)['start'].tolist() + pd.DataFrame(edges)['end'].tolist()
    edges.set_index(pd.Series(edges_start_list), inplace=True)
    edges.set_index('start', inplace=True)
    nodes = list(dict.fromkeys(nodes)) # remove duplicate
    nodes = pd.DataFrame(nodes,columns=['node'])
    nodes = nodes.set_index('node')
    nodes.sort_index(inplace = True)
    heuristics = pd.DataFrame(pd.read_csv('./heuristic.csv'),columns=['node', str(end)]).set_index('node')
    heuristics.sort_index(inplace = True)
    nodes['heuristics'] = heuristics[str(end)]
    visited = {}
    previous = {}
    distance = []

    nodes['visited'] = 0
    nodes['previous'] = -1
    nodes['g'] = 99999999999
    
    #print(" ===== edges table =====\n",edges)
    #print(" ===== nodes table =====\n",nodes)
    #print(" ===== heuristics table =====\n",heuristics)
    
    
    distance.append([start, 0, heuristics.at[start,str(end)], 0])
    nodes.at[str(start), 'visited'] = 1
    nodes.at[str(start), 'previous'] = -1
    num_visited = 0
    #global dist,accu_time,for_loop_time,sort_time,mid_time,nodes_query_time,edges_query_time,dist,done
    nodes_row,nodes_col = nodes.shape
    nodes_index = nodes.index
    #print("number of nodes: ",nodes_row)
    accu_time = 0.0
    for_loop_time = 0.0
    sort_time = 0.0
    mid_time = 0.0
    nodes_query_time = 0.0
    edges_query_time = 0.0
    t_pool = []
    #manager = Manager() # Lock 的 manager
    
    done = 0
    lock1 = threading.Lock()
    lock2 = threading.Lock()
    incumbent_cost = 99999999999
    while not done:
        #print(done)
        if(len(distance) == 0):
            continue
        t_pool.append(threading.Thread(target = job))
        t_pool[-1].start()
        #job()
    for i, t in enumerate(t_pool):
        t_pool[i].join()
    '''print("===== Performance =====")
    print("while loop time: ",t13 - t12)
    print("    sorting time", sort_time)
    print("    middle time", mid_time)
    print("      nodes query time", nodes_query_time)
    print("      edges query time", edges_query_time)
    print("    for loop time: ",for_loop_time)
    print("=======================")'''
    num_visited = nodes['visited'].sum() - 1
    path = []
    path.append(end)
    current_node = end
    t6 = time.time()
    nodes.at[start,'previous'] = -1
    while nodes.at[current_node,'previous'] != -1:
        path.append(nodes.at[current_node,'previous'])
        current_node = nodes.at[current_node,'previous']

    path.reverse()
    t7 = time.time()
    return path, dist, num_visited
    
    

In [19]:
# Don't change this part.
# Show the result of A* search
#global dist,accu_time,for_loop_time,sort_time,mid_time,nodes_query_time,edges_query_time,dist,done,sort_time
#global edges, nodes, heuristics, visited, previous

import time
st = time.time()
astar_path, astar_dist, astar_visited = astar(start, end)
en = time.time()

print(f'The number of nodes in the path found by A* search: {len(astar_path)}')
print(f'Total distance of path found by A* search: {astar_dist} m')
print(f'The number of visited nodes in A* search: {astar_visited}\n')
print("\nExecution time: ",en-st)

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(astar_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap



 ===== edges table =====
                    end  distance  speed limit
start                                        
26059311    3256115046    53.141         50.0
26059311    4429332928    11.548         46.4
26059311    4419128653    14.588         50.0
26059312    3256115020    33.469         50.0
26059312     420547128    52.411         50.0
...                ...       ...          ...
8513026829  1072288414    43.595         59.3
8513026829  7537285727    32.068         59.3
8513027209  2399106487    58.160         46.4
8513027209  2399106471     4.288         46.4
8540463858  4413408590    42.700         59.3

[23654 rows x 3 columns]
 ===== nodes table =====
               heuristics  visited  previous            g
node                                                    
26059311     8461.891002        0        -1  99999999999
26059312     8748.606356        0        -1  99999999999
26059315     8911.062591        0        -1  99999999999
26059321     9113.111144        0      